In [145]:
import os
import pandas as pd

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
analyze_path = os.path.join(parent_dir, "utils")

os.chdir(analyze_path)

In [ ]:
import bnlearn as bn

import matplotlib.pyplot as plt
plt.rcParams['font.family'] = ['Arial Unicode Ms']
# plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']

In [ ]:
from utils import read_data
combined_data = read_data()

In [151]:
select_group = [

    # 號誌
    '號誌-號誌種類名稱', 
    # '號誌-號誌動作名稱',

    # 車道劃分
    '車道劃分設施-分道設施-快車道或一般車道間名稱',
    '車道劃分設施-分道設施-快慢車道間名稱', '車道劃分設施-分道設施-路面邊線名稱',

    # 大類別
    # '車輛撞擊部位大類別名稱-最初',
    # '事故類型及型態大類別名稱', '車道劃分設施-分向設施大類別名稱',
    # '道路型態大類別名稱',
    # '車輛撞擊部位子類別名稱-最初',
    '事故類型及型態子類別名稱', '車道劃分設施-分向設施子類別名稱',
    '道路型態子類別名稱',

    # 其他
    '速限-第1當事者',
    '道路類別-第1當事者-名稱',

    # 設施
    'youbike_100m_count', 'mrt_100m_count', 'parkinglot_100m_count',
    
    # 駕駛、行人行為
    '肇因研判子類別名稱-主要'
    ]

In [158]:
data = combined_data[select_group].copy()
data['facility'] = data[['youbike_100m_count', 'mrt_100m_count', 'parkinglot_100m_count']].apply(
    lambda row: '1' if (row > 0).any() else '0', axis=1
)
data.drop(columns=['youbike_100m_count', 'mrt_100m_count', 'parkinglot_100m_count'], inplace=True)

max_speed = data['速限-第1當事者'].max()
bins = range(0, int(max_speed) + 11, 10)

data['速限-第1當事者'] = pd.cut(
    data['速限-第1當事者'],
    bins=bins,
    right=False, 
    include_lowest=True,
    labels=[f"{i}-{i+9}" for i in bins[:-1]]
)

In [ ]:
for i in data.columns:
    print(i, len(data[i].unique()))

In [159]:
# 學哪些變數之間有邊，結果是一個DAG
model = bn.structure_learning.fit(data, methodtype='hc', scoretype='bic', bw_list_method='edges',
                                  # 肇因對於事故類型一定是上游
                                   black_list=[('事故類型及型態子類別名稱', '肇因研判子類別名稱-主要')])
# 計算每個節點的 條件機率表 (CPT, Conditional Probability Table)
model_param = bn.parameter_learning.fit(model, data)
# 計算邊緣強度，如果p小於顯著就是有相關
model_independence = bn.independence_test(model_param, data, test='chi_square', prune=True)

[bnlearn] >Warning: Computing DAG with 11 nodes can take a very long time!
[bnlearn] >Computing best DAG using [hc]
[bnlearn] >Set scoring type at [bic]
[bnlearn] >Filter edges based on black_list/white_list
[bnlearn] >Compute structure scores for model comparison (higher is better).
[bnlearn] >Parameter learning> Computing parameters using [bayes]
[bnlearn] >Converting [<class 'pgmpy.base.DAG.DAG'>] to BayesianNetwork model.
[bnlearn] >Converting adjmat to BayesianNetwork.
[bnlearn] >CPD of 號誌-號誌種類名稱:
+-----------------------------+-----------+
| 號誌-號誌種類名稱(無號誌)              | 0.589508  |
+-----------------------------+-----------+
| 號誌-號誌種類名稱(行車管制號誌)           | 0.260669  |
+-----------------------------+-----------+
| 號誌-號誌種類名稱(行車管制號誌(附設行人專用號誌)) | 0.0990085 |
+-----------------------------+-----------+
| 號誌-號誌種類名稱(閃光號誌)             | 0.0508145 |
+-----------------------------+-----------+
[bnlearn] >CPD of 道路型態子類別名稱:
+--------------------+-----+
| 肇因研判子類別名稱-主要       | ... |
+--------

In [160]:
model_independence['independence_test']

source                   target  stat_test  p_value  \
0                 號誌-號誌種類名稱                道路型態子類別名稱       True      0.0   
1                 號誌-號誌種類名稱                 速限-第1當事者       True      0.0   
2                 號誌-號誌種類名稱             肇因研判子類別名稱-主要       True      0.0   
3                 號誌-號誌種類名稱                 facility       True      0.0   
4                 道路型態子類別名稱         車道劃分設施-分向設施子類別名稱       True      0.0   
5                  速限-第1當事者      車道劃分設施-分道設施-快慢車道間名稱       True      0.0   
6                  速限-第1當事者            道路類別-第1當事者-名稱       True      0.0   
7              肇因研判子類別名稱-主要  車道劃分設施-分道設施-快車道或一般車道間名稱       True      0.0   
8              肇因研判子類別名稱-主要             事故類型及型態子類別名稱       True      0.0   
9              肇因研判子類別名稱-主要                道路型態子類別名稱       True      0.0   
10                 facility       車道劃分設施-分道設施-路面邊線名稱       True      0.0   
11  車道劃分設施-分道設施-快車道或一般車道間名稱       車道劃分設施-分道設施-路面邊線名稱       True      0.0   
12  車道劃分設施-分道設施-快車道或一般車道間名稱         車道劃分設施-分向設施子類別名稱       True      0.0   
13  車道劃分設施-分道設施-快車道或一般車道間名稱                 速限-第1當事者       True      0.0   
14       車道劃分設施-分道設施-路面邊線名稱      車道劃分設施-分道設施-快慢車道間名稱       True      0.0   
15         車道劃分設施-分向設施子類別名稱      車道劃分設施-分道設施-快慢車道間名稱       True      0.0   
16         車道劃分設施-分向設施子類別名稱       車道劃分設施-分道設施-路面邊線名稱       True      0.0   
17         車道劃分設施-分向設施子類別名稱                 速限-第1當事者       True      0.0   
18         車道劃分設施-分向設施子類別名稱            道路類別-第1當事者-名稱       True      0.0   
19            道路類別-第1當事者-名稱       車道劃分設施-分道設施-路面邊線名稱       True      0.0   
20            道路類別-第1當事者-名稱                 facility       True      0.0   

      chi_square   dof  
0   9.129016e+04    45  
1   2.617484e+04    33  
2   1.573517e+05   336  
3   6.364917e+03     3  
4   3.019276e+04    75  
5   2.499225e+04    44  
6   3.000588e+05    88  
7   2.449759e+04   448  
8   1.324956e+06  3248  
9   2.882847e+05  1680  
10  7.339435e+03     1  
11  2.451346e+04     4  
12  1.761403e+05    20  
13  5.466413e+04    44  
14  4.241070e+03     4  
15  6.866424e+04    20  
16  5.512274e+04     5  
17  6.541057e+04    55  
18  1.882078e+04    40  
19  1.654795e+04     8  
20  9.834494e+03     8

In [161]:
bn.get_parents(model['model_edges'])

{'肇因研判子類別名稱-主要': ['號誌-號誌種類名稱'],
 '速限-第1當事者': ['號誌-號誌種類名稱', '車道劃分設施-分道設施-快車道或一般車道間名稱', '車道劃分設施-分向設施子類別名稱'],
 '道路型態子類別名稱': ['號誌-號誌種類名稱', '肇因研判子類別名稱-主要'],
 'facility': ['號誌-號誌種類名稱', '道路類別-第1當事者-名稱'],
 '車道劃分設施-分向設施子類別名稱': ['車道劃分設施-分道設施-快車道或一般車道間名稱', '道路型態子類別名稱'],
 '車道劃分設施-分道設施-路面邊線名稱': ['車道劃分設施-分道設施-快車道或一般車道間名稱',
  '車道劃分設施-分向設施子類別名稱',
  '道路類別-第1當事者-名稱',
  'facility'],
 '車道劃分設施-分道設施-快慢車道間名稱': ['車道劃分設施-分道設施-路面邊線名稱', '車道劃分設施-分向設施子類別名稱', '速限-第1當事者'],
 '道路類別-第1當事者-名稱': ['車道劃分設施-分向設施子類別名稱', '速限-第1當事者'],
 '事故類型及型態子類別名稱': ['肇因研判子類別名稱-主要'],
 '車道劃分設施-分道設施-快車道或一般車道間名稱': ['肇因研判子類別名稱-主要'],
 '號誌-號誌種類名稱': []}

In [ ]:
from matplotlib import rcParams
rcParams['font.sans-serif'] = ['Microsoft JhengHei']
rcParams['axes.unicode_minus'] = False

# G = bn.plot(model, interactive=False, node_color="#36AA5B", edge_labels=None)
bn.plot(model_independence, interactive=False, edge_labels='pvalue', 
        params_static={'layout': 'spring_layout', 'font_color': "#974848", 'edge_color': "#974848"})

In [163]:
import numpy as np
import pandas as pd
import networkx as nx
import plotly.graph_objects as go

def draw_bn_plotly(model, alpha=0.05, layout_algo=""):
    edges = [(str(u), str(v)) for u, v in model['model_edges']]
    df = model['independence_test'][['source','target','p_value']].copy()
    df['source'] = df['source'].astype(str); df['target'] = df['target'].astype(str)
    p_map = {(s,t):p for s,t,p in df.itertuples(index=False, name=None)}
    p_map.update({(t,s):p for (s,t),p in list(p_map.items())})

    G = nx.DiGraph()
    G.add_edges_from(edges)

    pos = (nx.spring_layout(G, seed=42) if layout_algo=="spring"
           else nx.kamada_kawai_layout(G))

    # nodes
    deg = dict(G.degree()); mdeg = max(deg.values()) if deg else 1
    node_x, node_y, node_text, node_size = [], [], [], []
    for n in G.nodes():
        x,y = pos[n]
        node_x.append(x); node_y.append(y)
        node_text.append(f"{n}<br>degree: {deg.get(n,0)}")
        node_size.append(10 + 25*(deg.get(n,1)/mdeg))

    node_trace = go.Scatter(
        x=node_x, y=node_y, mode='markers+text',
        text=[str(n) for n in G.nodes()], textposition="top center",
        hovertext=node_text, hoverinfo="text",
        marker=dict(size=node_size, line=dict(width=1), color="#0B4772")
    )

    # edges
    edge_traces = []
    for u,v in G.edges():
        x0,y0 = pos[u]; x1,y1 = pos[v]
        p = p_map.get((u,v), np.nan)
        if np.isnan(p):
            width, dash, color = 1.0, "dot", "#999"
            tip = f"{u} → {v}<br>p-value: N/A"
        else:
            w = -np.log10(max(p, 1e-300))
            width = 1 # 1 + 0.8*min(10, w)
            sig = (p <= alpha)
            dash = "solid" if sig else "dot"
            color = "#396bac" if sig else "#53a3dd"
            tip = f"{u} → {v}<br>p-value: {p:.3e}"

        edge_traces.append(go.Scatter(
            x=[x0, x1], y=[y0, y1],
            mode='lines',
            hoverinfo='text', text=[tip],
            line=dict(width=width, color=color, dash=dash)
        ))

    # 箭頭
    annotations = []
    for u,v in G.edges():
        x0,y0 = pos[u]; x1,y1 = pos[v]
        annotations.append(dict(
            ax=x0, ay=y0, x=x1, y=y1,
            xref="x", yref="y", axref="x", ayref="y",
            showarrow=True, arrowhead=3, arrowsize=1.2, opacity=0.8
        ))

    fig = go.Figure(data=edge_traces + [node_trace],
        layout=go.Layout(
            template=None, showlegend=False,
            hovermode='closest',
            margin=dict(l=10, r=10, t=10, b=10),
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            annotations=annotations,
            width=1000, height=600,
        )
    )
    return fig

draw_bn_plotly(model_independence, alpha=0.01, layout_algo='')

In [ ]:
# Conditional Probability Distributions (CPDs)
CPDs = bn.print_CPD(model_param)

## 肇因研判依賴於號誌種類

In [ ]:
dfprob_cause = CPDs['肇因研判子類別名稱-主要'].sort_values('p', ascending=False)
# dfprob_cause['p'] = round(dfprob_cause['p'], 4)
# dfprob_cause.head(30)

parent  = '號誌-號誌種類名稱'
child = '肇因研判子類別名稱-主要'

counts = (data.groupby([child, parent]).size().reset_index(name='n'))

# merge cpd for p and n
dfprob_cause_counts = (
    dfprob_cause
    .merge(counts, on=[child, parent], how='left')
    .sort_values('p', ascending=False)
)
# filter low frequency
filtered = dfprob_cause_counts[dfprob_cause_counts['n'] >= 30]
filtered['p'] = round(filtered['p'], 4)
filtered.head(30)


肇因研判子類別名稱-主要         號誌-號誌種類名稱       p        n
0             違反閃光號誌              閃光號誌  0.2517   4083.0
1          未保持行車安全距離  行車管制號誌(附設行人專用號誌)  0.1420   4490.0
2   有號誌路口，轉彎車未讓直行車先行            行車管制號誌  0.1319  10978.0
3          未保持行車安全距離            行車管制號誌  0.1298  10808.0
4           其他未依規定讓車              閃光號誌  0.1153   1870.0
5   有號誌路口，轉彎車未讓直行車先行  行車管制號誌(附設行人專用號誌)  0.1143   3612.0
6           其他不當駕車行為               無號誌  0.1090  20516.0
7           其他不當駕車行為  行車管制號誌(附設行人專用號誌)  0.1016   3212.0
8          未保持行車安全距離               無號誌  0.1001  18843.0
9           其他不當駕車行為            行車管制號誌  0.0952   7926.0
10    恍神、緊張、心不在焉分心駕駛               無號誌  0.0845  15901.0
11             闖紅燈直行            行車管制號誌  0.0804   6695.0
12  有號誌路口，轉彎車未讓直行車先行              閃光號誌  0.0725   1175.0
13    恍神、緊張、心不在焉分心駕駛            行車管制號誌  0.0689   5732.0
14           左轉彎未依規定  行車管制號誌(附設行人專用號誌)  0.0688   2173.0
15           左轉彎未依規定            行車管制號誌  0.0685   5701.0
16         未保持行車安全距離              閃光號誌  0.0626   1014.0
17          其他不當駕車行為              閃光號誌  0.0624   1010.0
18  無號誌路口，支線道未讓幹線道先行               無號誌  0.0620  11666.0
19             闖紅燈直行  行車管制號誌(附設行人專用號誌)  0.0582   1837.0
20          尚未發現肇事因素               無號誌  0.0562  10573.0
21          其他未依規定讓車               無號誌  0.0548  10318.0
22          尚未發現肇事因素  行車管制號誌(附設行人專用號誌)  0.0548   1730.0
23           左轉彎未依規定              閃光號誌  0.0542    877.0
24  無號誌路口，轉彎車未讓直行車先行               無號誌  0.0524   9866.0
25           右轉彎未依規定  行車管制號誌(附設行人專用號誌)  0.0518   1635.0
26          尚未發現肇事因素            行車管制號誌  0.0502   4177.0
27          起步時未注意安全               無號誌  0.0495   9326.0
28    恍神、緊張、心不在焉分心駕駛  行車管制號誌(附設行人專用號誌)  0.0494   1561.0
29           右轉彎未依規定            行車管制號誌  0.0461   3840.0

## 事故類型依賴於肇因研判

In [201]:
dfprob_cause = CPDs['事故類型及型態子類別名稱'].sort_values('p', ascending=False)
# dfprob_cause['p'] = round(dfprob_cause['p'], 4)
# dfprob_cause.head(30)

parent = '肇因研判子類別名稱-主要'
child  = '事故類型及型態子類別名稱'

counts = (data.groupby([child, parent]).size().reset_index(name='n'))

# merge cpd for p and n
dfprob_cause_counts = (
    dfprob_cause
    .merge(counts, on=[child, parent], how='left')
    .sort_values('p', ascending=False)
)

filtered = dfprob_cause_counts[dfprob_cause_counts['n'] >= 30]
filtered['p'] = round(filtered['p'], 4)
filtered.head(30)

事故類型及型態子類別名稱                 肇因研判子類別名稱-主要       p        n
0         路口交岔撞             無號誌路口，左方車未讓右方車先行  0.8748   5802.0
1         穿越道路中                  未依標誌或標線穿越道路  0.8431   1072.0
2            其他                    開啟或關閉車門不當  0.8177   1864.0
3         穿越道路中             未依規定行走地下道、天橋穿越道路  0.7884    121.0
4           倒車撞                       倒車未依規定  0.7665   2139.0
5         穿越道路中                車輛未依規定暫停讓行人先行  0.7665   2522.0
6            其他                  未待乘客安全上下而開車  0.7579     39.0
7         穿越道路中             未依號誌或手勢指揮(示)穿越道路  0.7529    169.0
8            其他                      裝載貨物不穩妥  0.7436    271.0
9         穿越道路中                  穿越道路未注意左右來車  0.7295    554.0
10          撞動物                         動物竄出  0.7276    482.0
11           其他                       其他裝載不當  0.7206    161.0
12           側撞             有號誌路口，轉彎車未讓直行車先行  0.7200  11738.0
13           追撞                    未保持行車安全距離  0.7081  24900.0
14           側撞             無號誌路口，轉彎車未讓直行車先行  0.7081   7284.0
15        路口交岔撞                       違反閃光號誌  0.6821   2878.0
16           其他                     裝載未盡安全措施  0.6632     52.0
17        路口交岔撞             無號誌路口，支線道未讓幹線道先行  0.6430   7965.0
18           追撞  使用車輛自動駕駛或先進駕駛輔助系統設備(裝置)不符規定  0.6369     34.0
19           側撞              違反二段式左(右)轉標誌(線)  0.6332    617.0
20           其他                     違規(臨時)停車  0.6282    347.0
21        路口交岔撞                        闖紅燈直行  0.6192   5328.0
22           側撞                  違反禁止左轉、右轉標誌  0.6107    330.0
23           側撞                      左轉彎未依規定  0.6083   9419.0
24           其他                       裝卸貨物不當  0.6070     33.0
25           其他                 其他引起事故之疏失或行為  0.5962    287.0
26           側撞                      右轉彎未依規定  0.5858   5629.0
28           側撞                   闖紅燈左轉(或迴轉)  0.5598    701.0
29         同向擦撞                    未保持行車安全間隔  0.5534   6673.0
30           其他                  車輛未停妥滑動致生事故  0.5534     45.0

## 快車道、一般車道依賴於肇因研判

In [202]:
dfprob_cause = CPDs['車道劃分設施-分道設施-快車道或一般車道間名稱'].sort_values('p', ascending=False)

parent = '肇因研判子類別名稱-主要'
child  = '車道劃分設施-分道設施-快車道或一般車道間名稱'

counts = (data.groupby([child, parent]).size().reset_index(name='n'))

# merge cpd for p and n
dfprob_cause_counts = (
    dfprob_cause
    .merge(counts, on=[child, parent], how='left')
    .sort_values('p', ascending=False)
)

filtered = dfprob_cause_counts[dfprob_cause_counts['n'] >= 30]
filtered['p'] = round(filtered['p'], 4)
filtered.head(30)

車道劃分設施-分道設施-快車道或一般車道間名稱                         肇因研判子類別名稱-主要       p  \
0                   未繪設車道線                     無號誌路口，左方車未讓右方車先行  0.9251   
1                   未繪設車道線                                未靠右行駛  0.8708   
2                   未繪設車道線                     無號誌路口，支線道未讓幹線道先行  0.8248   
3                   未繪設車道線                               違反閃光號誌  0.7798   
4                   未繪設車道線                    峻狹坡路會車，下坡車未讓上坡車先行  0.7670   
5                   未繪設車道線                     無號誌路口，轉彎車未讓直行車先行  0.7518   
6                 車道線(附標記)          使用車輛自動駕駛或先進駕駛輔助系統設備(裝置)不符規定  0.7200   
7                   未繪設車道線                     無號誌路口，少線道未讓多線道先行  0.7126   
8                   未繪設車道線                               倒車未依規定  0.6725   
9                   未繪設車道線                          違反禁止超車標誌(線)  0.6645   
10                  未繪設車道線                        車輛未依規定暫停讓行人先行  0.6493   
11                  未繪設車道線                     有號誌路口，轉彎車未讓直行車先行  0.6476   
12                  未繪設車道線                                 違規超車  0.6376   
14                  未繪設車道線                          穿越道路未注意左右來車  0.6367   
15                  未繪設車道線                           闖紅燈左轉(或迴轉)  0.6320   
16                  未繪設車道線                                 逆向行駛  0.6310   
17                  未繪設車道線                             未依規定使用燈光  0.6264   
18                  未繪設車道線                             其他未依規定讓車  0.6206   
19                  未繪設車道線                                闖紅燈右轉  0.6200   
20                  未繪設車道線                             起步時未注意安全  0.6123   
21                  未繪設車道線                                 動物竄出  0.6122   
22                  未繪設車道線                                闖紅燈直行  0.6100   
23                  未繪設車道線                          違反其他標誌(線)禁制  0.6084   
24                  未繪設車道線                              左轉彎未依規定  0.6022   
25                  未繪設車道線  相關跡證不足且無具體影像紀錄，當事人各執一詞，經分析後無法釐清肇事原因  0.6000   
26                  未繪設車道線                               未依規定減速  0.5986   
27                  未繪設車道線                          在道路上嬉戲或奔走不定  0.5972   
28                  未繪設車道線                          車輛未停妥滑動致生事故  0.5958   
29                  未繪設車道線                              酒醉(後)駕駛  0.5911   
30                  未繪設車道線                             違反禁止進入標誌  0.5896   

          n  
0    6134.0  
1    2041.0  
2   10215.0  
3    3289.0  
4      38.0  
5    7732.0  
6      37.0  
7     715.0  
8    1875.0  
9     658.0  
10   2135.0  
11  10557.0  
12   1663.0  
14    482.0  
15    790.0  
16   2156.0  
17    493.0  
18  10280.0  
19    292.0  
20   7330.0  
21    404.0  
22   5247.0  
23   2719.0  
24   9323.0  
25   2614.0  
26   3252.0  
27     71.0  
28     47.0  
29   1840.0  
30     40.0

## 道路型態依賴於肇因研判、號誌種類

In [203]:
dfprob_cause = CPDs['道路型態子類別名稱'].sort_values('p', ascending=False)

parent = '肇因研判子類別名稱-主要'
parent2 = '號誌-號誌種類名稱'
child  = '道路型態子類別名稱'

counts = (data.groupby([child, parent, parent2]).size().reset_index(name='n'))

# merge cpd for p and n
dfprob_cause_counts = (
    dfprob_cause
    .merge(counts, on=[child, parent, parent2], how='left')
    .sort_values('p', ascending=False)
)

filtered = dfprob_cause_counts[dfprob_cause_counts['n'] >= 30]
filtered['p'] = round(filtered['p'], 4)
filtered.head(30)

道路型態子類別名稱                                       肇因研判子類別名稱-主要  \
0        四岔路                                   無號誌路口，左方車未讓右方車先行   
1         直路                                        未待乘客安全上下而開車   
2         直路                                          開啟或關閉車門不當   
3         直路                                        在道路上嬉戲或奔走不定   
4         直路                                        違反禁止迴轉或迴車標誌   
5         直路                                         停車操作時未注意安全   
6         直路                                             變換車道不當   
7        四岔路                                   無號誌路口，左方車未讓右方車先行   
8        四岔路                                   無號誌路口，左方車未讓右方車先行   
9         直路                                  操作、觀看行車輔助或娛樂性顯示設備   
10        直路                           道路設施(備)、植栽或其他裝置，倒塌或掉(斷)落   
11        直路                                打瞌睡或疲勞駕駛(包括連續駕車8小時)   
12        直路                                飲食、抽(點)菸、拿(撿)物品分心駕駛   
13        直路                                          車輪脫落或輪胎爆裂   
14        圓環                                         行經圓環未依規定讓車   
15        直路                                        未依標誌或標線穿越道路   
18        直路                                      患病或服用藥物(疲勞)駕駛   
19        直路                                            裝載貨物不穩妥   
20        直路                                               動物竄出   
21        直路                                           違規(臨時)停車   
22       四岔路  未避讓(跟隨、併駛、超車)消防、救護、警備、工程救險車、毒性化學物質災害事故應變車等執行緊急任務車   
23       四岔路                                             違反閃光號誌   
24        直路                        使用車輛自動駕駛或先進駕駛輔助系統設備(裝置)不符規定   
25       四岔路                                   無號誌路口，轉彎車未讓直行車先行   
26        直路                                       其他引起事故之疏失或行為   
27       三岔路                                   無號誌路口，轉彎車未讓直行車先行   
28        直路                                   未依規定行走地下道、天橋穿越道路   
30        直路                                           起步時未注意安全   
33       四岔路                                   無號誌路口，支線道未讓幹線道先行   
34        直路                                        穿越道路未注意左右來車   

           號誌-號誌種類名稱       p       n  
0                無號誌  0.9202  6000.0  
1                無號誌  0.9127    32.0  
2                無號誌  0.8786  1730.0  
3                無號誌  0.8537    82.0  
4                無號誌  0.8511   326.0  
5                無號誌  0.8440   314.0  
6                無號誌  0.8253  3878.0  
7             行車管制號誌  0.8228    42.0  
8               閃光號誌  0.8208    46.0  
9                無號誌  0.8178    72.0  
10               無號誌  0.8168    76.0  
11               無號誌  0.8158   700.0  
12               無號誌  0.8149   282.0  
13               無號誌  0.8063   200.0  
14               無號誌  0.8052   149.0  
15               無號誌  0.8034   797.0  
18               無號誌  0.7868   882.0  
19               無號誌  0.7777   203.0  
20               無號誌  0.7739   452.0  
21               無號誌  0.7737   342.0  
22            行車管制號誌  0.7654    36.0  
23              閃光號誌  0.7650  3125.0  
24               無號誌  0.7613    32.0  
25  行車管制號誌(附設行人專用號誌)  0.7613    32.0  
26               無號誌  0.7602   295.0  
27               無號誌  0.7545  7445.0  
28               無號誌  0.7538    95.0  
30               無號誌  0.7392  6895.0  
33               無號誌  0.7339  8563.0  
34               無號誌  0.7323   498.0